# Master Project: Vehicle Detection & Tracking

Welcome to your complete project! This notebook is split into two parts:
1.  **Part 1: The Basics.** We will manually track **one** specific sequence step-by-step.
2.  **Part 2: Advanced Analysis.** We will automate the process to test **Sunny, Rainy, and Night** conditions.

**Note:** Each step is defined in a small cell. Run them one by one!

# PART 1: The Basics (Single Sequence)

### 1.1 Install Libraries
We need `ultralytics` for AI, `opencv` for video, and `lapx` for tracking math.

In [1]:
%pip install ultralytics opencv-python lapx

Note: you may need to restart the kernel to use updated packages.


### 1.2 Import OS
Used to navigate files on your computer.

In [2]:
import os

### 1.3 Import OpenCV
Used to read images and save videos.

In [3]:
import cv2

### 1.4 Import Glob
Used to find lists of files.

In [4]:
import glob

### 1.5 Import Plotting
Used to show images on screen.

In [5]:
import matplotlib.pyplot as plt

### 1.6 Import YOLO
Safety Check: Import the AI model.

In [6]:
from ultralytics import YOLO

---

### 1.7 Define Data Path
Where is the image folder stored?

In [7]:
# The folder containing all the sequence subfolders
IMAGES_ROOT_FOLDER = "/Volumes/X9 Pro/Resume Project/Multi-Object-Tracking/archive-2/DETRAC-Images/DETRAC-Images"

### 1.8 Define Specific Sequence
We will pick `MVI_20011` for our manual test.

In [8]:
seq_name = "MVI_20011"
single_seq_path = os.path.join(IMAGES_ROOT_FOLDER, seq_name)

### 1.9 Check if it exists
Validation step.

In [9]:
if os.path.exists(single_seq_path):
    print("Folder found.")
else:
    print("Folder NOT found. Check paths.")

Folder found.


---

### 1.10 List Images
Find all JPGs in that folder.

In [10]:
images = []
for f in os.listdir(single_seq_path):
    if f.endswith('.jpg'):
        images.append(f)

### 1.11 Sort Images
Ensure they are in order (1, 2, 3...) not random.

In [11]:
images = sorted(images)
print(f"Found {len(images)} images.")

Found 664 images.


---

### 1.12 Read First Image
Load the first frame to check size.

In [12]:
first_image_path = os.path.join(single_seq_path, images[0])
img = cv2.imread(first_image_path)

### 1.13 Get Dimensions
Get height and width.

In [13]:
height, width, layers = img.shape
print(f"Size: {width} x {height}")

Size: 960 x 540


---

### 1.14 Setup Video Writer
Prepare to create `part1_demo.mp4`.

In [14]:
video_name = "part1_demo.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(video_name, fourcc, 25, (width, height))

### 1.15 Write Video Loop
Stitch images into video.

In [15]:
print("Making video... please wait.")
for filename in images:
    path = os.path.join(single_seq_path, filename)
    frame = cv2.imread(path)
    video_writer.write(frame)

video_writer.release()
print("Video created.")

Making video... please wait.
Video created.


---

### 1.16 Load AI Model
Use YOLOv8 Nano.

In [16]:
model = YOLO('yolov8n.pt')

### 1.17 Define Tracking Arguments
Prepare settings: Classes 2,3,5,7 are vehicles.

In [17]:
track_config = {
    "source": video_name,
    "save": True,
    "tracker": "bytetrack.yaml",
    "classes": [2, 3, 5, 7],
    "project": "runs/detect",
    "name": "part1_result" 
}

### 1.18 Run Tracking
Execute the AI.

In [18]:
print("Running AI Tracking...")
results = model.track(
    source=track_config["source"],
    save=track_config["save"],
    tracker=track_config["tracker"],
    classes=track_config["classes"],
    project=track_config["project"],
    name=track_config["name"],
    exist_ok=True # Allows overwriting existing folder
)
print("Tracking Done.")

Running AI Tracking...

WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/664) /Volumes/X9 Pro/Resume Project/Multi-Object-Tracking/part1_demo.mp4: 384x640 14 cars, 44.2ms
video 1/1 (frame 2/664) /Volumes/X9 Pro/Resume Project/Multi-Object-Tracking/part1_demo.mp4: 384x640 13 cars, 31.9ms
video 1/1 (frame 3/664) /Volumes/X9 Pro/Resume Project/Multi-Object-Tracking/part1_demo.mp4: 384x640 14 cars, 28.9ms
video 1/1 (frame 4/664) /Volumes/X9 Pro/Resume Project/Multi-Object-Tracking/part1_d

# PART 2: Advanced Weather Analysis
Now we categorize sequences by weather using XML files.

### 2.1 Import XML Parser
To read the metadata.

In [19]:
import xml.etree.ElementTree as ET

### 2.2 Define Annotation Path
Location of XML files.

In [20]:
XML_ROOT = "/Volumes/X9 Pro/Resume Project/Multi-Object-Tracking/archive-2/DETRAC-Train-Annotations-XML/DETRAC-Train-Annotations-XML"

### 2.3 Get List of XML Files
Find all .xml files.

In [21]:
xml_files = glob.glob(os.path.join(XML_ROOT, "*.xml"))
print(f"Found {len(xml_files)} XML files.")

Found 60 XML files.


### 2.4 Initialize Weather Map
Storage for our categories.

In [22]:
weather_map = {'sunny': [], 'rainy': [], 'cloudy': [], 'night': []}

### 2.5 Scan XMLs
Loop through files and check weather tags.

In [23]:
print("Scanning files (this is fast)...")
for xml_file in xml_files:
    try:
        tree = ET.parse(xml_file)
        root = tree.getroot()
        name = root.attrib.get('name')
        
        for child in root:
            # We look for 'sence_weather' inside the xml
            if 'sence_weather' in child.attrib:
                weather = child.attrib['sence_weather'].lower()
                if weather in weather_map:
                    weather_map[weather].append(name)
    except:
        pass

Scanning files (this is fast)...


### 2.6 Show Weather Docs
See how many we found.

In [24]:
for w, seqs in weather_map.items():
    print(f"{w.capitalize()}: {len(seqs)} sequences.")

Sunny: 15 sequences.
Rainy: 10 sequences.
Cloudy: 19 sequences.
Night: 16 sequences.


---

### 2.7 Define Automation Function
A function to run our pipeline on any sequence.

In [25]:
def run_weather_test(seq_name, weather_label):
    print(f"\nrunning TEST: {weather_label} ({seq_name})")
    
    # 1. Setup Path
    path = os.path.join(IMAGES_ROOT_FOLDER, seq_name)
    
    # 2. Get Images
    imgs = sorted([i for i in os.listdir(path) if i.endswith('.jpg')])
    if not imgs: return
    
    # 3. Read Frame 1 for size
    frame1 = cv2.imread(os.path.join(path, imgs[0]))
    h, w, _ = frame1.shape
    
    # 4. Create MP4
    vid_name = f"test_{weather_label}.mp4"
    out = cv2.VideoWriter(vid_name, cv2.VideoWriter_fourcc(*'mp4v'), 25, (w, h))
    
    print("Creating video...")
    for i in imgs:
        out.write(cv2.imread(os.path.join(path, i)))
    out.release()
    
    # 5. Run YOLO Tracking
    print("Tracking...")
    model.track(
        source=vid_name, 
        project="runs/detect", 
        name=f"result_{weather_label}", 
        save=True,
        classes=[2,3,5,7],
        exist_ok=True
    )
    print("Done.")

### 2.8 Select Test Cases
Pick one from each list.

In [26]:
sunny_seq = weather_map['sunny'][0] if weather_map['sunny'] else None
rainy_seq = weather_map['rainy'][0] if weather_map['rainy'] else None
night_seq = weather_map['night'][0] if weather_map['night'] else None

### 2.9 Run Sunny Test

In [27]:
if sunny_seq:
    run_weather_test(sunny_seq, "sunny")


running TEST: sunny (MVI_20051)
Creating video...
Tracking...

WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/906) /Volumes/X9 Pro/Resume Project/Multi-Object-Tracking/test_sunny.mp4: 384x640 10 cars, 1 bus, 40.0ms
video 1/1 (frame 2/906) /Volumes/X9 Pro/Resume Project/Multi-Object-Tracking/test_sunny.mp4: 384x640 10 cars, 1 bus, 29.3ms
video 1/1 (frame 3/906) /Volumes/X9 Pro/Resume Project/Multi-Object-Tracking/test_sunny.mp4: 384x640 10 cars, 2 buss, 30.4ms
video 1/1 (frame 4/906

### 2.10 Run Rainy Test

In [28]:
if rainy_seq:
    run_weather_test(rainy_seq, "rainy")


running TEST: rainy (MVI_63544)
Creating video...
Tracking...

WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1160) /Volumes/X9 Pro/Resume Project/Multi-Object-Tracking/test_rainy.mp4: 384x640 12 cars, 2 buss, 66.2ms
video 1/1 (frame 2/1160) /Volumes/X9 Pro/Resume Project/Multi-Object-Tracking/test_rainy.mp4: 384x640 11 cars, 2 buss, 34.1ms
video 1/1 (frame 3/1160) /Volumes/X9 Pro/Resume Project/Multi-Object-Tracking/test_rainy.mp4: 384x640 11 cars, 3 buss, 32.6ms
video 1/1 (frame 

### 2.11 Run Night Test

In [29]:
if night_seq:
    run_weather_test(night_seq, "night")


running TEST: night (MVI_39851)
Creating video...
Tracking...

WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1420) /Volumes/X9 Pro/Resume Project/Multi-Object-Tracking/test_night.mp4: 384x640 6 cars, 1 bus, 51.1ms
video 1/1 (frame 2/1420) /Volumes/X9 Pro/Resume Project/Multi-Object-Tracking/test_night.mp4: 384x640 6 cars, 1 bus, 39.1ms
video 1/1 (frame 3/1420) /Volumes/X9 Pro/Resume Project/Multi-Object-Tracking/test_night.mp4: 384x640 5 cars, 1 bus, 33.1ms
video 1/1 (frame 4/1420

---

### 3. Conclusion & Occlusion Check

You now have videos in `runs/detect/result_sunny`, `_rainy`, and `_night`.

**To check Occlusion Performance:**
1.  Open each output video.
2.  Find a moment where one car passes another (hiding it).
3.  Check the ID boxes.
    *   **Good:** ID stays same (e.g. #5 before, #5 after).
    *   **Bad:** ID changes (e.g. #5 before, #12 after).

**End of Notebook**